In [1]:
import pandas as pd
from google.cloud import bigquery
import os
from google.cloud.bigquery.client import Client
import numpy as np
import re
import string
import nltk
nltk.download('stopwords')
from sklearn import preprocessing
from datetime import datetime
import math
from decimal import *
import random

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
dayP=4

In [4]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'G:/hedden/asu-msba-free-trial-conversion-credentials.json'
bq_client = Client()
client = bigquery.Client()
projectID= 'infusionsoft-looker-poc'
#standardSQLjob_config = bigquery.QueryJobConfig()
##read Usage data from Bigquery, with the trial time all equal to 14 days
job_config = bigquery.QueryJobConfig()
query_params = [
    bigquery.ScalarQueryParameter('dayP', 'INT64', dayP)
]
job_config.query_parameters = query_params
job_config.use_legacy_sql = False##Standard Sql
query="""SELECT
  U.*,F.trial_date as trial_date
FROM
  `infusionsoft-looker-poc.asu_msba_free_trial_conversion.CONFIDENTIAL_usage_data` AS U JOIN
  (SELECT app_name,trial_date,end_day FROM (SELECT app_name,CAST(trial_date AS date) as trial_date,
  IF(CAST(trial_date AS date)<CAST(new_customer_date AS date),if(CAST(new_customer_date AS date)>DATE_ADD(CAST(trial_date AS date),INTERVAL @dayP DAY),DATE_ADD(CAST(trial_date AS date),INTERVAL @dayP DAY),CAST(new_customer_date AS date)),DATE_ADD(CAST(trial_date AS date),INTERVAL @dayP DAY)) as end_day
  FROM `infusionsoft-looker-poc.asu_msba_free_trial_conversion.CONFIDENTIAL_free_trail_apps_table`) group by app_name,trial_date,end_day) AS F
  ON U.appname=F.app_name
WHERE
  U.date>=F.trial_date
  AND U.date<=F.end_day
ORDER BY U.appname,U.date"""
read= client.query(query,location='US',job_config=job_config)
UsageData=read.result().to_dataframe()
##make a copy of the USage data
UD=UsageData.copy()
DFreeTrial= pd.read_gbq('SELECT * FROM asu_msba_free_trial_conversion.CONFIDENTIAL_free_trail_apps_table ',projectID)
DFree=DFreeTrial.copy()
droplist=['tl471','wd410','mw416','kw563','ov450','jl500']
DFree=DFree[~DFree['app_name'].isin(droplist)]
UD=UD[~UD['appname'].isin(droplist)]


C:\ProgramData\Anaconda3\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: FutureWarning: The default value for dialect is changing to "standard" in a future version of pandas-gbq. Pass in dialect="legacy" to disable this warning.


In [5]:
def oldcustomer(x):## get the old customer'Nl==1'
    old=[]
    for index,row in x.iterrows():
        if pd.isna(row['new_customer_date']):
            old.append(0)
        elif row['new_customer_date']<row['trial_date']:
            old.append(1)
        else:
            old.append(0)
    x['Nl']=old
    return x

In [6]:
def phase(x):## get the phase number
    if pd.isna(x) or x=='None': return 0
    else:
        m=x.split(' ')[1]
        return int(m)

In [7]:
def dropDup(df,cat): ## delete the duplicate caused by join
    dfN = df.groupby(['app_name'])[[cat]].apply(lambda x: x.sort_values([cat], ascending=False).iloc[0, :])
    return dfN
def totaltar(df):##delete the duplicate caused by join and drop duplicates
    df = df[['app_name', 'is_free_trial_initiated','trial_date', 'new_customer_date','first_contact_on', 'contact_lead_source', 'contact_phase', 'lead_lead_source','lead_converted_date']]
    df=df.drop_duplicates()
    AP = df[['app_name', 'is_free_trial_initiated', 'trial_date', 'new_customer_date']]
    AP=AP.drop_duplicates()
    Data = df.drop(['is_free_trial_initiated', 'trial_date'], axis=1)
    DFL = []
    for cat in ['first_contact_on', 'contact_lead_source', 'contact_phase', 'lead_lead_source', 'lead_converted_date']:
        dfN = dropDup(Data,cat)
        DFL.append(dfN)
    DFreee = pd.concat(DFL, axis=1)
    DFreee['app_name'] = DFreee.index
    DFreee.reset_index(drop=True,inplace=True)
    AP.drop_duplicates(inplace=True)
    df = AP.merge(DFreee, left_on=['app_name'], right_on=['app_name'])
    return df
FR= totaltar(DFree)

In [8]:
from nltk.corpus import stopwords
def soursetrans(text):## change the leadsource and contact source
    sw = stopwords.words('english');text = text.lower();patten1 = string.digits;patten2 = string.punctuation
    regex = re.compile(r"[%s%s]" % (patten1, patten2));text = regex.sub(" ", text);regex = re.compile(r"\s+")
    text = regex.sub(" ", text);textlist = text.split()
    textclean = [w for w in textlist if w.lower() not in sw]
    text1 = ['google', 'web', 'email', 'softwareadvice', 'forbes', 'doubleyoursales', 'www', 'com', 'emailfooter']
    text2 = ['infusionsoft', 'freetrial', 'demo', 'direct', 'content', 'free', 'search']
    text3 = ['partner', 'referral', 'recruiting', 'lead']
    text4 = ['zunknown']
    text5 = ['marketo', 'campaign']
    text6 = ['salesline', 'mobile', 'onlinechat', 'offlinechat', 'chat', 'us', 'marketing', 'salesforce']
    text7 = ['facebook', 'twitter', 'social']
    output = [];retA = []
    for x in textclean:
        if x not in output:
            output.append(x)
    for i in output:
        if i in text:
            retA.append(i)
    if retA == []:
        retA = output
    group = 'other'
    for i in retA:
        if i in text1: group = 'advertise'
        if i in text2: group = 'directsearch'
        if i in text3: group = 'refer'
        if i in text4: group = 'unknown'
        if i in text5: group = 'marketo_campaign'
        if i in text6: group = 'chat'
        if i in text7: group = 'socialmedia'
    return group

In [9]:
def outline(x):
    if x<=-13 or x==None: return -14
    else:return x
def floatc(x):
    if pd.isna(x):
        return np.nan
    else:
        return float(x)
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

In [10]:
def Free_process(df1):#process 
    df1.loc[:,'contact_phase']=df1['contact_phase'].apply(phase)
    df1[['contact_lead_source','lead_lead_source']]=df1[['contact_lead_source','lead_lead_source']].fillna('zunknown')
    for cate in ['contact_lead_source','lead_lead_source']:
        df1.loc[:,cate] = df1[cate].apply(lambda x: soursetrans(x))
    Tnow = datetime.now()
    for date in ['first_contact_on','lead_converted_date']:
        df1.loc[:,date] = pd.to_datetime(df1[date])
        df1.loc[:,date].fillna(Tnow, inplace=True)
        df1.loc[:,date] = ((pd.to_datetime(df1['trial_date'].dt.date) - pd.to_datetime(df1[date].dt.date)) / np.timedelta64(1, 'D')).astype('int')
        df1.loc[:,date]=df1[date].apply(outline)
    df1.loc[:,'is_free_trial_initiated']=df1['is_free_trial_initiated'].apply(lambda x:x+0)
    df1=df1[df1['is_free_trial_initiated']==1]
    df1.drop(['is_free_trial_initiated'],inplace=True,axis=1)
    df1['trial_date']=pd.to_datetime(df1['trial_date'].dt.date)
    df1=oldcustomer(df1)
    df1=df1[df1['Nl']==0]
    df1.loc[:,'trial_date']=pd.to_datetime(df1['trial_date'].dt.date)
    df1=df1.drop_duplicates()
    df1.reset_index(drop=True, inplace=True)
    df1.drop(['Nl'],axis=1,inplace=True)
    df1['contact_phase'] = df1['contact_phase'].astype('int64')
    df1['target'] = df1['new_customer_date'].apply(lambda x:0 if pd.isna(x) else 1 )
    numb=[x for x in df1.columns.tolist() if x not in ['app_name','trial_date','new_customer_date','account_id','contact_id','lead_lead_source','contact_lead_source']]
    MMSF = preprocessing.MinMaxScaler(feature_range=(0, 1), copy=True)
    df1[numb] = pd.DataFrame(MMSF.fit_transform(df1[numb]), index=df1.index)
    ohe = preprocessing.OneHotEncoder(sparse=False, dtype=int, handle_unknown='ignore')
    cat = ['lead_lead_source','contact_lead_source']
    Xcat = pd.DataFrame(ohe.fit_transform(df1[cat]), columns=ohe.get_feature_names(), index=df1.index)
    dfN= pd.concat([df1, Xcat], axis=1)
    dff=dfN[[x for x in dfN.columns.tolist() if x not in cat]]
    return dff,MMSF,ohe

In [11]:
NNN=FR.copy()
Freee,MMSF,OHE=Free_process(NNN)

C:\ProgramData\Anaconda3\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\ProgramData\Anaconda3\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

In [12]:
UD1=UD.drop(['NUM_CONTACTS'],axis=1)

In [13]:
## preprocess and minmax scale the USage data
def Usage_process(df2):
    df2=df2.drop_duplicates()
    df2.loc[:,'free_email']=df2['free_email'].apply(lambda x:x+0)
    df2.loc[:,'invoice_amount'] = df2['invoice_amount'].apply(floatc)
    df2=df2.fillna(0)
    df2.loc[:,'trial_date']=pd.to_datetime(df2['trial_date'])
    MMSdf = preprocessing.MinMaxScaler(feature_range=(0, 1), copy=True)
    numb2 = [x for x in df2.columns.tolist() if x not in ['appname', 'trial_date', 'date']]
    df2[numb2] = pd.DataFrame(MMSdf.fit_transform(df2[numb2]), index=df2.index)
    return df2,MMSdf
UD2,MMU=Usage_process(UD1)


C:\ProgramData\Anaconda3\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\ProgramData\Anaconda3\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [14]:
def get_usedat(UD2,dayP):
    ZeroN=np.zeros((14,72))
    Usage={}
    for name,group in UD2.groupby(['appname','trial_date']):
        G=group[[x for x in group.columns.tolist() if x not in ['appname','trial_date']]].groupby(['date']).agg('mean')
        L=G.shape[0]
        New=np.concatenate((G.values, ZeroN), axis=0)
        TG=New[0:dayP]
        Usage[name]=[name,TG,L]
    Usagedata=pd.DataFrame.from_dict(Usage, orient='index',columns=['name','Gdata','len'])
    return Usagedata
Usagedata=get_usedat(UD2,4)

In [24]:
Free={}
for name,group in Freee.groupby(['app_name','trial_date']):
    if name not in Usagedata['name'].values.tolist():
        continue
    target=int(group['target'].values[0])
    K=group[[x for x in group.columns.tolist() if x not in ['app_name','trial_date','target','is_free_trial_initiated','new_customer_date']]].values
    Len=group.shape[0]
    Us=Usagedata[Usagedata['name']==name]['Gdata'].values[0]
    Ul=Usagedata[Usagedata['name']==name]['len'].values[0]
    Free[name]=[name,K,target,Len,Us,Ul]
Freedata=pd.DataFrame.from_dict(Free, orient='index',columns=['name','Fdata','target','Flen','Usage','Ul'])

import torch
from torch import nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.autograd import Variable

In [16]:
import tensorflow as tf

C:\ProgramData\Anaconda3\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [25]:
Freedata['target'].sum()

1420

In [29]:
def split(df):
    Con=df[df['target']==1]
    Ncon=df[df['target']==0]
    TRc=Con.sample(frac=0.9)
    TSc=Con[~Con.index.isin(TRc.index)]
    TRn=Ncon.sample(frac=0.2)
    TSn=Ncon[~Ncon.index.isin(TRn.index)]
    Train=pd.concat([TRc.sample(frac=2.5,replace=True),TRn])
    Test=pd.concat([TSc.sample(frac=2.5,replace=True),TSn])
    Train['target']=Train['target'].apply(lambda x:[0,1] if x==1 else [1,0])
    Test['target']=Test['target'].apply(lambda x:[0,1] if x==1 else [1,0])
    Test=pd.concat([TSc.sample(frac=2.5,replace=True),TSn])
    Train=Train.sort_values(by=['Ul'],ascending=False)
    Test=Test.sort_values(by=['Ul'],ascending=False)
    Train_x=np.array(Train['Usage'].tolist())
    Train_y=np.array(Train['target'].tolist()).reshape(Train['target'].shape[0],-1)
    Train_S=Train['Ul'].tolist()
    Test_x=np.array(Test['Usage'].tolist())
    Test_y=np.array(Test['target'].tolist()).reshape(Test['target'].shape[0],-1)
    Test_S=Test['Ul'].tolist()        
    return Train_x,Train_y,Train_S,Test_x,Test_y,Test_S
Train_x,Train_y,Train_S,Test_x,Test_y,Test_S=split(Freedata)

In [32]:
learning_rate = 0.01    # 学习率
n_steps = dayP            # LSTM 展开步数（时序持续长度）
n_inputs = 72           # 输入节点数
n_hiddens = 64         # 隐层节点数
n_layers = 2            # LSTM layer 层数
n_classes = 2          # 输出节点数（分类数目）
_batch_size=128

In [33]:
# tensor placeholder
with tf.name_scope('inputs'):
    x = tf.placeholder(tf.float32, [None, n_steps, n_inputs], name='x_input')     # input
    y = tf.placeholder(tf.float32, [None, n_classes], name='y_input')               # output
    keep_prob = tf.placeholder(tf.float32, name='keep_prob_input')           # keep how not dropout
    batch_size = tf.placeholder(tf.int32, [], name='batch_size_input')
    x_lengths = [0]*_batch_size
# weights and biases
with tf.name_scope('weights'):
    Weights = tf.Variable(tf.truncated_normal([n_hiddens, n_classes],stddev=0.1), dtype=tf.float32, name='W')
    tf.summary.histogram('output_layer_weights', Weights)
with tf.name_scope('biases'):
    biases = tf.Variable(tf.random_normal([n_classes]), name='b')
    tf.summary.histogram('output_layer_biases', biases)


In [34]:
# RNN structure
def RNN_LSTM(x,x_lengths,Weights, biases,):
    x = tf.reshape(x, [-1, n_steps, n_inputs])
    # LSTM cell
    # cell dropout
    def attn_cell():
        lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hiddens,state_is_tuple=True)
        with tf.name_scope('lstm_dropout'):
            return tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=keep_prob)
    # attn_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=keep_prob)
    # [attn_cell() for _ in range(n_layers)]
    enc_cells = []
    for i in range(0, n_layers):
        enc_cells.append(attn_cell())
    with tf.name_scope('lstm_cells_layers'):
        mlstm_cell = tf.contrib.rnn.MultiRNNCell(enc_cells, state_is_tuple=True)
    # 0 state
    _init_state = mlstm_cell.zero_state(batch_size, dtype=tf.float32)
    # dynamic_rnn
    outputs, states = tf.nn.dynamic_rnn(mlstm_cell, x, sequence_length=x_lengths,initial_state=_init_state, dtype=tf.float32, time_major=False)
    #return tf.matmul(outputs[:,-1,:], Weights) + biases
    return tf.nn.softmax(tf.matmul(outputs[:,-1,:], Weights) + biases)


In [35]:
with tf.name_scope('output_layer'):
    pred = RNN_LSTM(x, x_lengths, Weights, biases)
    tf.summary.histogram('outputs', pred)
# cost
with tf.name_scope('loss'):
    #cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred),reduction_indices=[1]))
    tf.summary.scalar('loss', cost)
# optimizer
with tf.name_scope('train'):
    train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
# accuarcy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
with tf.name_scope('accuracy'):
    accuracy = tf.metrics.accuracy(labels=tf.argmax(y, axis=1), predictions=tf.argmax(pred, axis=1))[1]
    tf.summary.scalar('accuracy', accuracy)

merged = tf.summary.merge_all()

init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [36]:
with tf.Session() as sess:
    sess.run(init)
    # training
    step = 1
    Nbatch=math.ceil(int(Train_x.shape[0])/_batch_size)
    for i in range(Nbatch ):
        batch_x, batch_y,Sequence= Train_x[i*_batch_size:(i+1)*_batch_size,],Train_y[i*_batch_size:(i+1)*_batch_size,],Train_S[i*_batch_size:(i+1)*_batch_size]
        sess.run(train_op, feed_dict={x:batch_x, y:batch_y, keep_prob:0.5, batch_size:_batch_size})
        if (i + 1) % 100 == 0:
            loss = sess.run(cost, feed_dict={x:batch_x, y:batch_y,x_lengths:Sequence, keep_prob:1.0, batch_size:_batch_size})
            acc = sess.run(accuracy, feed_dict={x:batch_x, y:batch_y, keep_prob:1.0, batch_size:_batch_size})
            print('Iter: %d' % ((i+1) * _batch_size), '| train loss: %.6f' % loss, '| train accuracy: %.6f' % acc)
            train_result = sess.run(merged, feed_dict={x:batch_x, y:batch_y, keep_prob:1.0, batch_size:_batch_size})
            test_result = sess.run(merged, feed_dict={x:test_x, y:test_y, keep_prob:1.0, batch_size:test_x.shape[0]})
            train_writer.add_summary(train_result,i+1)
            test_writer.add_summary(test_result,i+1)

    print("Optimization Finished!")
    # prediction
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x:test_x, y:test_y, keep_prob:1.0, batch_size:test_x.shape[0]}))

InvalidArgumentError: assertion failed: [Expected shape for Tensor output_layer/rnn/sequence_length:0 is ] [118] [ but saw shape: ] [128]
	 [[node output_layer/rnn/Assert/Assert (defined at <ipython-input-34-2934260ffa4f>:20)  = Assert[T=[DT_STRING, DT_INT32, DT_STRING, DT_INT32], summarize=3, _device="/job:localhost/replica:0/task:0/device:CPU:0"](output_layer/rnn/All, output_layer/rnn/Assert/Assert/data_0, output_layer/rnn/stack, output_layer/rnn/Assert/Assert/data_2, output_layer/rnn/Shape_1)]]

Caused by op 'output_layer/rnn/Assert/Assert', defined at:
  File "C:\ProgramData\Anaconda3\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\Anaconda3\lib\site-packages\traitlets\confi

class LSTMRNN(object):
    def __init__(self, n_steps, input_size, output_size, cell_size, batch_size,Seq_length):
        self.seq=Seq_length
        self.n_steps = n_steps
        self.input_size = input_size
        self.output_size = output_size
        self.cell_size = cell_size
        self.batch_size = batch_size
        with tf.name_scope('inputs'):
            self.xs = tf.placeholder(tf.float32, [None, n_steps, input_size], name='xs')
            self.ys = tf.placeholder(tf.float32, [None, n_steps, output_size], name='ys')
        with tf.variable_scope('in_hidden'):
            self.add_input_layer()
        with tf.variable_scope('LSTM_cell'):
            self.add_cell()
        with tf.variable_scope('out_hidden'):
            self.add_output_layer()
        with tf.name_scope('cost'):
            self.compute_cost()
        with tf.name_scope('train'):
            self.train_op = tf.train.AdamOptimizer(LR).minimize(self.cost)

    def add_input_layer(self,):
        Ws_in = self._weight_variable([self.input_size, self.cell_size])
        # bs (cell_size, )
        bs_in = self._bias_variable([self.cell_size,])
        # l_in_y = (batch * n_steps, cell_size)
        with tf.name_scope('Wx_plus_b'):
            l_in_y = tf.matmul(l_in_x, Ws_in) + bs_in
        # reshape l_in_y ==> (batch, n_steps, cell_size)
        self.l_in_y = tf.reshape(l_in_y, [-1, self.n_steps, self.cell_size], name='2_3D')

    def add_cell(self):
        lstm_cell = tf.contrib.rnn.BasicLSTMCell(self.cell_size, forget_bias=1.0, state_is_tuple=True)
        with tf.name_scope('initial_state'):
            self.cell_init_state = lstm_cell.zero_state(self.batch_size, dtype=tf.float32)
        self.cell_outputs, self.cell_final_state = tf.nn.dynamic_rnn(
            lstm_cell, self.l_in_y, initial_state=self.cell_init_state, time_major=False)

    def add_output_layer(self):
        # shape = (batch * steps, cell_size)
        l_out_x = tf.reshape(self.cell_outputs, [-1, self.cell_size], name='2_2D')
        Ws_out = self._weight_variable([self.cell_size, self.output_size])
        bs_out = self._bias_variable([self.output_size, ])
        # shape = (batch * steps, output_size)
        with tf.name_scope('Wx_plus_b'):
            self.pred = tf.matmul(l_out_x, Ws_out) + bs_out

    def compute_cost(self):
        losses = tf.contrib.legacy_seq2seq.sequence_loss_by_example(
            [tf.reshape(self.pred, [-1], name='reshape_pred')],
            [tf.reshape(self.ys, [-1], name='reshape_target')],
            [tf.ones([self.batch_size * self.n_steps], dtype=tf.float32)],
            average_across_timesteps=True,
            softmax_loss_function=self.ms_error,
            name='losses'
        )
        with tf.name_scope('average_cost'):
            self.cost = tf.div(
                tf.reduce_sum(losses, name='losses_sum'),
                self.batch_size,
                name='average_cost')
            tf.summary.scalar('cost', self.cost)

    @staticmethod
    def ms_error(labels, logits):
        return tf.square(tf.subtract(labels, logits))

    def _weight_variable(self, shape, name='weights'):
        initializer = tf.random_normal_initializer(mean=0., stddev=1.,)
        return tf.get_variable(shape=shape, initializer=initializer, name=name)

    def _bias_variable(self, shape, name='biases'):
        initializer = tf.constant_initializer(0.1)
        return tf.get_variable(name=name, shape=shape, initializer=initializer)

##def split(df):
    Con=df[df['target']==1]
    Ncon=df[df['target']==0]
    TRc=Con.sample(frac=0.9)
    TSc=Con[~Con.index.isin(TRc.index)]
    TRn=Ncon.sample(frac=0.2)
    TSn=Ncon[~Ncon.index.isin(TRn.index)]
    Train=pd.concat([TRc.sample(frac=2.5,replace=True),TRn])
    Test=pd.concat([TSc.sample(frac=2.5,replace=True),TSn])
    Train=Train.sort_values(by=['Ul'],ascending=False)
    Test=Test.sort_values(by=['Ul'],ascending=False)
    Train_x=torch.from_numpy(np.array(Train['Usage'].tolist())).double()
    Train_y=torch.from_numpy(np.array(Train['target'].tolist()).reshape(Train['target'].shape[0],-1)).double()
    Train_S=Train['Ul'].tolist()
    Test_x=torch.from_numpy(np.array(Test['Usage'].tolist())).double()
    Test_y=torch.from_numpy(np.array(Test['target'].tolist()).reshape(Test['target'].shape[0],-1)).double()
    Test_S=Test['Ul'].tolist()          
    return Train_x,Train_y,Train_S,Test_x,Test_y,Test_S
Train_x,Train_y,Train_S,Test_x,Test_y,Test_S=split(Freedata)

EPOCH=3
BATCH_SIZE=10
TIME_STEP=14
INPUT_SIZE=72
LR=0.01

class RNN(nn.Module):
    def __init__(self, nb_layers=3,nb_lstm_units=80,input_size=72,batch_size=10):
        super(RNN,self).__init__()
        self.nb_lstm_layers = nb_layers
        self.nb_lstm_units = nb_lstm_units
        self.inputsize = input_size
        self.batch_size = batch_size
        self.nb_tags=2
        self.lstm = nn.LSTM(
            input_size=self.inputsize,
            hidden_size=self.nb_lstm_units,
            num_layers=self.nb_lstm_layers,
            batch_first=True,
        )
        self.hidden_to_tag = nn.Linear(self.nb_lstm_units,2)

    def init_hidden(self):
        # the weights are of the form (nb_layers, batch_size, nb_lstm_units)
        hidden_a = torch.randn(self.nb_lstm_layers, self.batch_size, self.nb_lstm_units)
        hidden_b = torch.randn(self.nb_lstm_layers, self.batch_size, self.nb_lstm_units)

        hidden_a = Variable(hidden_a)
        hidden_b = Variable(hidden_b)

        return (hidden_a, hidden_b)

    def forward(self, X,X_length):
        X=X.type(torch.float32)
        self.hidden = self.init_hidden()
        X=Variable(X)
        X=torch.nn.utils.rnn.pack_padded_sequence(X, X_length, batch_first=True)
        X,self.hidden= self.lstm(X, self.hidden)
        X, _= torch.nn.utils.rnn.pad_packed_sequence(X, batch_first=True)
        X = X.contiguous()
        X = X.view(-1, X.shape[2])
        X = self.hidden_to_tag(X)
        X = F.log_softmax(X, dim=1)
        X = X.view(batch_size, seq_len, self.nb_tags)
        Y_hat = X
        return Y_hat
rnn = RNN()


optimizer=torch.optim.Adam(rnn.parameters(),lr=LR)
loss_func=nn.CrossEntropyLoss()

n_batch=math.ceil(Train_x.shape[0]/BATCH_SIZE)
for epoch in range(1):
    for i in range(n_batch):
        local_X, local_y,Sequence= Train_x[i*10:(i+1)*10,],Train_y[i*10:(i+1)*10,],Train_S[i*10:(i+1)*10]
        output=rnn(local_X,X_length=Sequence)
        loss=loss_func(output,local_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if step % 50 == 0:
            test_output = rnn(Test_x,Seq=Test_S)                   # (samples, time_step, input_size)
            pred_y = torch.max(test_output, 1)[1].data.numpy()
            accuracy = float((pred_y == Test_y).astype(int).sum()) / float(Test_y.size)
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)